In [74]:
import pandas as pd
import numpy as np
import time
from datetime import datetime as dt, date, timedelta
from polygon import RESTClient
from secret import API_KEY

In [75]:
# Getting API key and client
api_key = API_KEY
client = RESTClient(api_key)

In [76]:
# Importing buy/sell signals
data = pd.read_csv('signals.csv')

In [77]:
# Creating columns for prices purchased and sold at 
data['call_prices'] = np.zeros(len(data))
data['put_prices'] = np.zeros(len(data))

In [78]:
# Setting ticker
underlying_ticker = 'AAPL'

In [79]:
def get_option_ticker(timestamp, price, type):
    # Fixing timestamps
    date = pd.to_datetime(timestamp).date()
    date_more = date + timedelta(weeks=1)

    try:
        # Initialize a list to collect all contract data
        options_list = []

        # Fetch options data
        options_data = client.list_options_contracts(
            underlying_ticker=underlying_ticker,
            expiration_date_lte=date_more,
            expiration_date_gte=date,
            strike_price_gte=(price),
            strike_price_lte=(price+1),
            contract_type=type,
            limit=1000,
            expired=True
        )

        # Loop through the options data and append each contract's data to options_list
        for option in options_data:
            options_list.append(option.__dict__)

        # Convert the list of options data to a DataFrame
        df = pd.DataFrame(options_list)

        # Finding closest ticker
        option_ticker = df['ticker'].iloc[0]

        return option_ticker
    except Exception as e:
        print(f'Error: {e}')

    return None

def get_option_data(timestamp, ticker):
    date = pd.to_datetime(timestamp).date()

    try:
        aggs = client.get_aggs(
                ticker=ticker,
                multiplier=1,
                timespan="minute",
                from_=date,
                to=date,
                limit=5000
            )
        
        df = pd.DataFrame([agg.__dict__ for agg in aggs])

        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df['timestamp'] = df['timestamp'].dt.tz_localize('UTC').dt.tz_convert('America/New_York')

        df['time'] = df['timestamp'].dt.time
        df['time'] = df['time'].apply(lambda t: t.strftime("%H:%M:%S"))
        
        return df
    except Exception as e:
        print(f"Error: {e}")

    return None


In [ ]:
for index, row in data.iterrows():
    time.sleep(24)
    if row['call_buy']:
        timestamp = row['timestamp']
        ticker = get_option_ticker(timestamp, row['close'], 'call')
        option_price_data = get_option_data(timestamp, ticker)

        for i in range(0,3):
            try:
                row_time = (pd.to_datetime(row['time']) + timedelta(minutes=i)).strftime("%H:%M:%S")


                buy_call_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]
                data.at[index, 'call_prices'] = buy_call_price
                
                break

            except:
                print('NO DATA')
                continue

    elif row['call_sell']:
        for i in range(0,3):
            try:
                row_time = (pd.to_datetime(row['time']) + timedelta(minutes=i)).strftime("%H:%M:%S")


                sell_call_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]
                data.at[index, 'call_prices'] = sell_call_price
                
                break

            except:
                print('NO DATA')
                continue

    elif row['put_buy']:
        timestamp = row['timestamp']
        ticker = get_option_ticker(timestamp, row['close'], 'put')
        option_price_data = get_option_data(timestamp, ticker)

        for i in range(0,3):
            try:
                row_time = (pd.to_datetime(row['time']) + timedelta(minutes=i)).strftime("%H:%M:%S")


                buy_put_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]
                data.at[index, 'put_prices'] = buy_put_price
                
                break

            except:
                print('NO DATA')
                continue

    elif row['put_sell']:
        for i in range(0,3):
            try:
                row_time = (pd.to_datetime(row['time']) + timedelta(minutes=i)).strftime("%H:%M:%S")


                sell_put_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]
                data.at[index, 'put_prices'] = sell_put_price
                
                break

            except:
                print('NO DATA')
                continue

KeyboardInterrupt: 

In [87]:
data.tail(10)

,timestamp,open,high,low,close,volume,vwap,transactions,date,time,...,ema_20,atr,upper_band,lower_band,call_buy,call_sell,put_buy,put_sell,call_prices,put_prices
2312,2024-11-05 14:31:00+00:00,223.08,223.88,223.00,223.5291,240930.0,223.5198,3381,2024-11-05,14:31:00,...,221.975504,0.212806,222.826728,221.124279,False,False,True,False,0.0,0.0
2313,2024-11-05 14:38:00+00:00,222.93,222.97,221.14,222.7600,226906.0,222.6449,3610,2024-11-05,14:38:00,...,222.431124,0.368824,223.906422,220.955827,False,False,False,True,0.0,0.0
2314,2024-11-06 11:52:00+00:00,223.12,223.12,222.80,222.8000,8499.0,223.0000,281,2024-11-06,11:52:00,...,223.321497,0.109727,223.760403,222.882591,True,False,False,False,0.0,0.0
2315,2024-11-06 11:55:00+00:00,223.03,223.08,222.60,222.6100,10152.0,222.7900,104,2024-11-06,11:55:00,...,223.140364,0.142088,223.708716,222.572012,False,True,False,False,0.0,0.0
2316,2024-11-06 11:56:00+00:00,222.62,222.67,222.41,222.6700,13459.0,222.5382,139,2024-11-06,11:56:00,...,223.095567,0.147984,223.687502,222.503633,True,False,False,False,0.0,0.0
2317,2024-11-06 11:57:00+00:00,222.70,222.98,222.59,222.9800,4304.0,222.7751,90,2024-11-06,11:57:00,...,223.084561,0.160084,223.724898,222.444223,False,True,False,False,0.0,0.0
2318,2024-11-06 12:08:00+00:00,223.61,223.99,223.61,223.9900,7894.0,223.8704,123,2024-11-06,12:08:00,...,223.277031,0.177928,223.988744,222.565318,False,False,True,False,0.0,0.0
2319,2024-11-06 12:14:00+00:00,223.80,223.80,223.66,223.6700,1970.0,223.7304,63,2024-11-06,12:14:00,...,223.517678,0.169843,224.197050,222.838305,False,False,False,True,0.0,0.0
2320,2024-11-06 14:28:00+00:00,223.01,223.01,222.30,222.5900,32075.0,222.6088,368,2024-11-06,14:28:00,...,223.606403,0.286258,224.751436,222.461370,True,False,False,False,0.0,0.0
2321,2024-11-06 14:32:00+00:00,223.09,224.57,223.08,224.2095,605913.0,224.0615,8597,2024-11-06,14:32:00,...,223.354557,0.486444,225.300332,221.408782,False,True,False,False,0.0,0.0
